# Test monthly average

First make up some data and calculate the averages, then try using my function, and see you get the same answer.

In [1]:
#import my functions
import sys
sys.path.append(sys.path[0]+'/../..')


import helpers.fileHandler as fh
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils._indexDefinitions as _index
from utils.timePeriod import averageForTimePeriod

In [2]:
import xarray
import numpy
import cftime
import pandas

I am going to make up some data, and put it in a pandas array first

In [3]:
#first make up some time period
dateRange=xarray.cftime_range(
    start=cftime.DatetimeNoLeap(1995,1,1),
    end=cftime.DatetimeNoLeap(1999,12,1),
    freq='M')

In [4]:
#put it into a pd Dataframe
testPd=pandas.DataFrame(index=dateRange)

testPd.index.name='time'

In [12]:
#add some rando data
for keys in _index.monthsOfInterest:
    testPd[keys]=5*numpy.random.rand(len(dateRange),1)

I am now going to calculate the averages in a really easy to understand way

In [13]:
#years of interest
expectedAv=pandas.DataFrame(index=[1995,1996,1997,1998])
expectedAv.index.name='year'

#for every index defined in the index file
for keys in _index.monthsOfInterest:
    answer = list()
    
    #grab the time period defined in the index file
    #this line is a bit lazy, i really should define these indeces some other way incase of systematic errors
    monthPeriod=_index.monthsOfInterest[keys].copy()

    #select the data for this index
    indexArray = testPd[keys]
    
    temp = 0
    #for every month in the data
    for monthIter in numpy.arange(0,len(indexArray),1):
        # a month in the time period of interest:
        if ((monthIter+1)>=monthPeriod[0] and (monthIter+1)<=monthPeriod[1]):
            temp += indexArray.values[monthIter]
        else:
        # not a month in the time period of interest
            if temp!=0:
                answer.append(temp/(monthPeriod[1]-monthPeriod[0]+1))
                temp = 0
                monthPeriod[0]+=12
                monthPeriod[1]+=12
    #save output for this index and move to the next one
    expectedAv[keys]=answer


expectedAv.round(5)

,nino34NoDetrend,nino4NoDetrend,westIONoDetrend,eastIONoDetrend,dmi,nino34,nino4,westIO,eastIO,sam
year,,,,,,,,,,
1995,2.44497,2.36956,1.96825,2.69948,2.09695,1.84111,2.62889,3.11660,2.81387,2.83194
1996,2.86742,2.45421,2.62511,3.22929,1.53736,2.00400,3.05116,2.45358,2.05932,2.46055
1997,3.24651,2.14931,3.34173,1.96905,2.93158,2.44532,2.56553,2.26143,1.87801,1.85157
1998,1.86576,3.09986,2.34779,2.46384,2.08002,2.47089,2.64109,2.27982,2.61062,3.10332


Calculate it using the function

In [14]:
#first cast it into an xarray
testXr = xarray.Dataset(testPd)

# do the calculation
calculatedAvs=averageForTimePeriod(testXr)

calculatedAvs.round(5)

,nino34NoDetrend,nino4NoDetrend,westIONoDetrend,eastIONoDetrend,dmi,nino34,nino4,westIO,eastIO,sam
year,,,,,,,,,,
1995,2.44497,2.36956,1.96825,2.69948,2.09695,1.84111,2.62889,3.11660,2.81387,2.83194
1996,2.86742,2.45421,2.62511,3.22929,1.53736,2.00400,3.05116,2.45358,2.05932,2.46055
1997,3.24651,2.14931,3.34173,1.96905,2.93158,2.44532,2.56553,2.26143,1.87801,1.85157
1998,1.86576,3.09986,2.34779,2.46384,2.08002,2.47089,2.64109,2.27982,2.61062,3.10332


In [15]:
numpy.isclose(calculatedAvs, expectedAv)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])